In [6]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score, confusion_matrix, ConfusionMatrixDisplay

In [ ]:
class LassoHateSpeech:
    """Lasso model using sentence embeddings to detect hate speech."""

    def __init__(self):
        """
        Initialize classifier with Lasso model and encoder.
        """
        self.encoder = SentenceTransformer("all-MiniLM-L6-v2")
        self.model = LogisticRegression(penalty="l1", solver="liblinear", max_iter=1000)

    def embed(self, texts: pd.Series) -> np.ndarray:
        """
        Embed text using SentenceTransformer.

        Args:
            texts (pd.Series): Input phrases.

        Returns:
            np.ndarray: Sentence embeddings.
        """
        return self.encoder.encode(texts.tolist(), show_progress_bar=False)

    def train(self, X: pd.Series, y: pd.Series) -> None:
        """
        Train the logistic regression model.

        Args:
            X (pd.Series): Text samples.
            y (pd.Series): Binary labels (0 or 1).
        """
        X_embed = self.embed(X)
        self.model.fit(X_embed, y)

    def predict_proba(self, X: pd.Series) -> pd.Series:
        """
        Predict probability of hate speech.

        Args:
            X (pd.Series): Input phrases.

        Returns:
            pd.Series: Predicted probabilities between 0 and 1.
        """
        X_embed = self.embed(X)
        proba = self.model.predict_proba(X_embed)[:, 1]
        return pd.Series(proba, index=X.index)

    def evaluate(self, X_test: pd.Series, y_test: pd.Series) -> None:
        """
        Print classification metrics using 0.25 probability threshold.

        Args:
            X_test (pd.Series): Validation text.
            y_test (pd.Series): True binary labels.
        """
        proba = self.predict_proba(X_test)
        y_pred = (proba >= 0.25).astype(int)
        print(classification_report(y_test, y_pred))


def main():
    """Train model and predict probabilities on test set."""
    train_df = pd.read_csv("../data/test_data.csv", sep=";")
    # test_df = pd.read_csv("../data/test.csv")

    train_df["isHate"] = train_df["isHate"].apply(lambda x: 1 if x >= 0.25 else 0)

    X_train, X_val, y_train, y_val = train_test_split(
        train_df["comment"], train_df["isHate"], test_size=0.2, random_state=42
    )

    model = LassoHateSpeech()
    model.train(X_train, y_train)
    model.evaluate(X_val, y_val)

    # test_probs = model.predict_proba(test_df["text"])
    # test_df["probability"] = test_probs
    # test_df.to_csv("../data/hate_speech_predictions.csv", index=False)

In [ ]:
train_df = pd.read_csv("../data/test_data.csv", sep=";")
train_df["isHate"] = train_df["isHate"].apply(lambda x: 1 if x >= 0.25 else 0)

X_train = train_df["comment"]
y_train = train_df["isHate"]

model = LassoHateSpeech()
model.train(X_train, y_train)

train_probs = model.predict_proba(X_train)

from sklearn.metrics import roc_curve, auc

fpr, tpr, _ = roc_curve(y_train, train_probs)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f"ROC curve (AUC = {roc_auc:.2f})")
plt.plot([0, 1], [0, 1], linestyle="--", color="gray")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve - Training Data")
plt.legend(loc="lower right")
plt.grid(True)
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: '../data/train_data.csv'

In [ ]:
precision, recall, _ = precision_recall_curve(y_train, train_probs)
avg_precision = average_precision_score(y_train, train_probs)

plt.figure(figsize=(8, 6))
plt.plot(recall, precision, label=f"Avg Precision = {avg_precision:.2f}")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curve - Training Data")
plt.legend(loc="upper right")
plt.grid(True)
plt.show()

In [ ]:
y_pred = (train_probs >= 0.25).astype(int)
cm = confusion_matrix(y_train, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Not Hate", "Hate"])
disp.plot(cmap="Blues")
plt.title("Confusion Matrix - Training Data")
plt.grid(False)
plt.show()